# introduction
## Problem Statement
We hypothesize to be a Start Up which has to open his headquarters either in Toronto or in New York. We want to ensure that our employers have the higher standard of life possible. Therefore, we want to assure that our employers live and work in a place which has enough places and facilities for everyone’s taste.

We want to understand the similarities and differences between Toronto and New York and find the best Neighborhood for a Start Up which wants to move its headquarters based on venues.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Toronto Dataset
The dateset being used is found at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.
The dataset is a list of Toronto's zipcodes which includes the boroughs and neighborhood names.

In [2]:
#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]

df_toronto.rename(columns = {"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)

#Combine the neighborhoods that exists in one postal code
df_toronto = df_toronto.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()

#Change unassigned Neighborhood to its Borough's name
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### join whith latitude/longitude information

In [3]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("http://cocl.us/Geospatial_data")

latlong.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)

#Join the Lat and Long dataframe to Neighborhoods dataframe
df_toronto.set_index("PostalCode")
latlong.set_index("PostalCode")
neighbor=pd.merge(df_toronto, latlong)
neighbor.head()

print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    )
)


Toronto has 11 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto, Canada.

In [4]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


### Create a map of Toronto with neighborhoods superimposed on top.

In [5]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Toronto Neighborhood
We suppose that our Start Up has to interface with financial institutions, so we want to locate it near the financial district. In Toronto the financial district is located in the Neighbourhood Downtown Toronto. So we will focus our analysis on this neighborhood.

In [7]:
Downtown_data = neighbor[neighbor['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [8]:
Downtown_data.shape

(18, 5)

In [9]:
# geographical coordinates of Scarborough
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto, CA are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Scarborough, CA are 43.6541737, -79.3808116451341.


In [10]:
# create map of Scarborough using latitude and longitude values
map_Downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_data['Latitude'], Downtown_data['Longitude'], Downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown)  
    
map_Downtown

# Explore Scarborough neighborhood in Toronto with Foursquare API

In [11]:
CLIENT_ID = 'JDK1RRIIAFVA1ZNSC2ZF031WV2DMIEZ5HZX0GPB2LLEDGKKN' 
CLIENT_SECRET = 'CAULNIG2OMNBJUAHLH3WA2RMXOE2VFIAHNYMQPW14ZDMJT4Q'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JDK1RRIIAFVA1ZNSC2ZF031WV2DMIEZ5HZX0GPB2LLEDGKKN
CLIENT_SECRET:CAULNIG2OMNBJUAHLH3WA2RMXOE2VFIAHNYMQPW14ZDMJT4Q


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

Downtown_venues = getNearbyVenues(names=Downtown_data['Neighborhood'],
                                   latitudes=Downtown_data['Latitude'],
                                   longitudes=Downtown_data['Longitude']
                                  )

Found 1284 venues in 18 neighborhoods.


In [15]:
print(Downtown_venues.shape)
Downtown_venues.head()

(1284, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [16]:
#Venues per Neighborhood
Downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,84,84,84,84,84,84
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [17]:
print('There are {} distinct venues in {} categories.'.format(
    len(Downtown_venues['Venue'].unique()),len(Downtown_venues['Venue Category'].unique())))

print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 754 distinct venues in 207 categories.
There are 207 uniques categories.



 # Analyze each Neighborhood

In [18]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_onehot['Neighborhood'] = Downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]

Downtown_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.03,0.000000,0.000000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.030000,0.000000,0.00000,0.01,0.020000,0.010000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.080000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.00,0.01,0.01,0.000000,0.010000,0.000000,0.00,0.

#### We Analyze each neighborhood with the top 5 venues

In [20]:
num_top_venues = 10

for hood in Downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4   Thai Restaurant  0.04
5             Hotel  0.03
6    Breakfast Spot  0.03
7               Gym  0.03
8        Restaurant  0.03
9  Asian Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3            Beer Bar  0.04
4                Café  0.04
5         Cheese Shop  0.04
6              Bakery  0.04
7  Seafood Restaurant  0.04
8          Steakhouse  0.04
9            Creperie  0.02


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3             Airport  0.07
4  Airport Food Court  0.07
5        Airport Gate  0.07
6               Plane  0.07
7         Coffee S

In [24]:
# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

We put this result into a pandas dataframe

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,Hotel,American Restaurant,Restaurant,Breakfast Spot
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Farmers Market,Steakhouse,Beer Bar,Bakery,Seafood Restaurant,Lounge
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Plane,Coffee Shop,Airport Gate,Harbor / Marina,Airport
3,"Cabbagetown,St. James Town",Coffee Shop,Park,Bakery,Café,Pub,Convenience Store,Italian Restaurant,Pizza Place,Restaurant,Deli / Bodega
4,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Café,Restaurant,Middle Eastern Restaurant,Spa,Bakery
5,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Bakery,Bar,Coffee Shop,Record Shop
6,Christie,Café,Grocery Store,Park,Convenience Store,Nightclub,Baby Store,Restaurant,Diner,Italian Restaurant,Coffee Shop
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Men's Store,Pub,Bubble Tea Shop,Gym
8,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Seafood Restaurant,Bakery,Steakhouse,Gym
9,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Gastropub,American Restaurant,Deli / Bodega,Bakery,Gym,Bar


In [26]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Harbord,University of Toronto
1st Most Common Venue                              Café
2nd Most Common Venue                               Bar
3rd Most Common Venue               Japanese Restaurant
4th Most Common Venue                         Bookstore
5th Most Common Venue                        Restaurant
6th Most Common Venue                            Bakery
7th Most Common Venue                Chinese Restaurant
8th Most Common Venue                          Beer Bar
9th Most Common Venue                        Beer Store
10th Most Common Venue                   Sandwich Place
Name: 11, dtype: object

# Cluster the Scarborough Neighborhood using k-means

In [27]:
# set number of clusters
kclusters = 3

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

### we create a new dataframe that includes the cluster as well as the top ten venues for each neighborhood

In [29]:

# add clustering labels
Downtown_data['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_data = Downtown_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_data.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Building,Women's Store,Discount Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Coffee Shop,Park,Bakery,Café,Pub,Convenience Store,Italian Restaurant,Pizza Place,Restaurant,Deli / Bodega
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Men's Store,Pub,Bubble Tea Shop,Gym
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Park,Pub,Bakery,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Brewery
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_data['Latitude'], Downtown_data['Longitude'], Downtown_data['Neighborhood'], Downtown_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Downtown Toronto Neighborhood Cluster
Now, we examine each cluster and determine the discriminating venue categories that distinguish each cluster. 


### Downtown Toronto Clusters 0, 1, 2 

In [31]:
Downtown_cluster_0 = Downtown_data.loc[Downtown_data['Cluster Labels'] == 0, Downtown_data.columns[[1] + list(range(4, Downtown_data.shape[1]))]]

Downtown_cluster_1 = Downtown_data.loc[Downtown_data['Cluster Labels'] == 1, Downtown_data.columns[[1] + list(range(4, Downtown_data.shape[1]))]]

Downtown_cluster_2 = Downtown_data.loc[Downtown_data['Cluster Labels'] == 2, Downtown_data.columns[[1] + list(range(4, Downtown_data.shape[1]))]]


In [32]:
Downtown_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.377529,0,Park,Playground,Trail,Building,Women's Store,Discount Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,Downtown Toronto,-79.367675,0,Coffee Shop,Park,Bakery,Café,Pub,Convenience Store,Italian Restaurant,Pizza Place,Restaurant,Deli / Bodega
3,Downtown Toronto,-79.360636,0,Coffee Shop,Café,Park,Pub,Bakery,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Brewery
4,Downtown Toronto,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop
5,Downtown Toronto,-79.375418,0,Hotel,Italian Restaurant,Coffee Shop,Café,Restaurant,Cocktail Bar,Bakery,Cosmetics Shop,Breakfast Spot,Beer Bar
6,Downtown Toronto,-79.373306,0,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Farmers Market,Steakhouse,Beer Bar,Bakery,Seafood Restaurant,Lounge
7,Downtown Toronto,-79.387383,0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Café,Restaurant,Middle Eastern Restaurant,Spa,Bakery
8,Downtown Toronto,-79.384568,0,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,Hotel,American Restaurant,Restaurant,Breakfast Spot
9,Downtown Toronto,-79.381752,0,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Bakery,Sporting Goods Shop,Pizza Place
10,Downtown Toronto,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Gastropub,American Restaurant,Deli / Bodega,Bakery,Gym,Bar


In [33]:
Downtown_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,-79.39442,1,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Plane,Coffee Shop,Airport Gate,Harbor / Marina,Airport


In [34]:
Downtown_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,-79.38316,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Men's Store,Pub,Bubble Tea Shop,Gym


# 6. Explore New York City Neighborhoods

In [35]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# transform data into a dataframe
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Data downloaded!


In [41]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [37]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


## Use Geolibrary to get the latitude and longitude of New York City

In [38]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [39]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### As for Toronto
we need to locate our Enterprise into the financial district, in order to facilitate communications between the Start Up and Financial stakeholders, in new york the financial district is in Manhattan

In [47]:
Manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
Manhattan_data.head(30)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [43]:
# Geographycal location
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [44]:
# create map of Manhattan using latitude and longitude values
map_Manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Manhattan_data['Latitude'], Manhattan_data['Longitude'], Manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Manhattan)  
    
map_Manhattan

Now, suppose that our Start Up is looking to work as near as possible whith its stakeholders, we will locate it in the center of financial district --> Financial District, Manhattan


In [49]:
Manhattan_data.loc[29, 'Neighborhood']

'Financial District'

In [50]:
#Long Island City Latitude and Longitude values

neighborhood_latitude = Manhattan_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Manhattan_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = Manhattan_data.loc[29, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Financial District are 40.76811265828733, -73.9588596881376.


# Top 100 venues in Financial District neighborhood within a radius of 1000 meters

In [61]:
# First, let's create the GET request URL named url.
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=JDK1RRIIAFVA1ZNSC2ZF031WV2DMIEZ5HZX0GPB2LLEDGKKN&client_secret=CAULNIG2OMNBJUAHLH3WA2RMXOE2VFIAHNYMQPW14ZDMJT4Q&v=20180605&ll=40.76811265828733,-73.9588596881376&radius=1000&limit=100'

In [62]:
#Send the GET request
results = requests.get(url).json()

In [63]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [64]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Whiskey & Wine Off 69,Liquor Store,40.767272,-73.959544
1,Up Thai,Thai Restaurant,40.769898,-73.957598
2,Cigar Inn,Smoke Shop,40.768776,-73.956222
3,sweetgreen,Salad Place,40.767128,-73.956846
4,Anthropologie,Women's Store,40.769296,-73.961085


In [65]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


# Analyze Each Neighborhood in Manhattan

In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

print('Done')

Done


In [72]:
Manhattan_venues = getNearbyVenues(names=Manhattan_data['Neighborhood'],
                                   latitudes=Manhattan_data['Latitude'],
                                   longitudes=Manhattan_data['Longitude'])

print('Done')

Found 3324 venues in 40 neighborhoods.
Done


In [74]:
Manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [75]:
#Venues per Neighborhood
Manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,47,47,47,47,47,47
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [76]:
print('There are {} distinct venues in {} categories.'.format(
    len(Manhattan_venues['Venue'].unique()),len(Manhattan_venues['Venue Category'].unique())))

There are 2887 distinct venues in 341 categories.


In [77]:
# one hot encoding
Manhattan_onehot = pd.get_dummies(Manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Manhattan_onehot['Neighborhood'] = Manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = Manhattan_onehot['Neighborhood']
Manhattan_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
Manhattan_onehot.insert(0, 'Neighborhood', neighbor)

Manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Gard

## Group by Neighborhood, and exame the frequency of the occurrence of venue

In [78]:
Manhattan_grouped = Manhattan_onehot.groupby('Neighborhood').mean().reset_index()
Manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Gard

In [79]:
# each neighborhood with the top 5 venues
num_top_venues = 5

for hood in Manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Manhattan_grouped[Manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0            Park  0.08
1     Coffee Shop  0.07
2           Hotel  0.05
3   Memorial Site  0.04
4  Clothing Store  0.03


----Carnegie Hill----
            venue  freq
0     Coffee Shop  0.07
1     Pizza Place  0.06
2             Bar  0.04
3            Café  0.04
4  Cosmetics Shop  0.04


----Central Harlem----
                  venue  freq
0    African Restaurant  0.06
1  Gym / Fitness Center  0.04
2    Seafood Restaurant  0.04
3                   Bar  0.04
4    Chinese Restaurant  0.04


----Chelsea----
                venue  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.05
2      Ice Cream Shop  0.05
3              Bakery  0.04
4           Nightclub  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1           Cocktail Bar  0.05
2    American Restaurant  0.04
3  Vietnamese Restaurant  0.04
4     Salon / Barbershop  0.04


----Civic Center----
                  venue  freq
0    Italian

In [80]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top venues for each neighborhood

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Manhattan_grouped['Neighborhood']

for ind in np.arange(Manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Italian Restaurant,Wine Shop,Gym,Women's Store,Clothing Store,Playground
1,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Bar,Café,French Restaurant,Japanese Restaurant,Grocery Store,Yoga Studio,Spa
2,Central Harlem,African Restaurant,Bar,Chinese Restaurant,Gym / Fitness Center,Seafood Restaurant,Public Art,Cosmetics Shop,Deli / Bodega,American Restaurant,Fried Chicken Joint
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,Hotel,American Restaurant,Seafood Restaurant,Art Gallery,Theater
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Bubble Tea Shop,Bakery,Spa,Ice Cream Shop,Dumpling Restaurant
5,Civic Center,Italian Restaurant,Gym / Fitness Center,French Restaurant,Hotel,Sandwich Place,Sporting Goods Shop,American Restaurant,Bakery,Cocktail Bar,Coffee Shop
6,Clinton,Theater,Gym / Fitness Center,Italian Restaurant,Hotel,American Restaurant,Wine Shop,Coffee Shop,Spa,Sandwich Place,Food Court
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Pizza Place,Thai Restaurant,Latin American Restaurant,French Restaurant,Steakhouse,Spanish Restaurant,Seafood Restaurant
8,East Village,Bar,Wine Bar,Ice Cream Shop,Pizza Place,Chinese Restaurant,Mexican Restaurant,Cocktail Bar,Ramen Restaurant,Speakeasy,Coffee Shop
9,Financial District,Coffee Shop,Hotel,Wine Shop,Gym,American Restaurant,Pizza Place,Steakhouse,Gym / Fitness Center,Event Space,Cocktail Bar


# Cluster the Manhattan Borough using K-Means

In [83]:
# set number of clusters
kclusters = 5

Manhattan_grouped_clustering = Manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(Manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([3, 3, 3, 0, 0, 0, 4, 1, 3, 3, 3, 0, 0, 1, 4, 1, 0, 4, 0, 0, 3, 1,
       0, 0, 3, 1, 3, 0, 1, 0, 2, 3, 4, 1, 0, 0, 3, 1, 0, 3], dtype=int32)

## We now create a dataframe that includes the cluster of each neighborhood

In [84]:
Manhattan_merged = Manhattan_data

# add clustering labels
Manhattan_merged['Cluster Labels'] = kmeans.labels_

Manhattan_merged = Manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Sandwich Place,Coffee Shop,Discount Store,Yoga Studio,Diner,Steakhouse,Supplement Shop,Shopping Mall,Seafood Restaurant,Tennis Stadium
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Bubble Tea Shop,Bakery,Spa,Ice Cream Shop,Dumpling Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Mobile Phone Shop,Bakery,Grocery Store,Supermarket,Deli / Bodega,Coffee Shop,Park,Clothing Store,Chinese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Café,Lounge,Pizza Place,Park,Bakery,Frozen Yogurt Shop,Deli / Bodega,Restaurant,Spanish Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Café,Pizza Place,Coffee Shop,Liquor Store,Indian Restaurant,Sushi Restaurant,Park,Deli / Bodega,Sandwich Place


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Manhattan_merged['Latitude'], Manhattan_merged['Longitude'], Manhattan_merged['Neighborhood'], Manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Manhattan Cluster Neighborhood
Now we examine each cluster and determine the discriminating venue categories that distinguish 
each cluster.

In [86]:
Manhattan_cluster_0 = Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 0, Manhattan_merged.columns[[1] + list(range(4, Manhattan_merged.shape[1]))]]

Manhattan_cluster_1 = Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 1, Manhattan_merged.columns[[1] + list(range(4, Manhattan_merged.shape[1]))]]

Manhattan_cluster_2 = Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 2, Manhattan_merged.columns[[1] + list(range(4, Manhattan_merged.shape[1]))]]

Manhattan_cluster_3 = Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 3, Manhattan_merged.columns[[1] + list(range(4, Manhattan_merged.shape[1]))]]

Manhattan_cluster_4 = Manhattan_merged.loc[Manhattan_merged['Cluster Labels'] == 4, Manhattan_merged.columns[[1] + list(range(4, Manhattan_merged.shape[1]))]]

In [87]:
Manhattan_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,0,Mexican Restaurant,Café,Lounge,Pizza Place,Park,Bakery,Frozen Yogurt Shop,Deli / Bodega,Restaurant,Spanish Restaurant
4,Hamilton Heights,0,Mexican Restaurant,Café,Pizza Place,Coffee Shop,Liquor Store,Indian Restaurant,Sushi Restaurant,Park,Deli / Bodega,Sandwich Place
5,Manhattanville,0,Italian Restaurant,Deli / Bodega,Park,Coffee Shop,Mexican Restaurant,Seafood Restaurant,Café,Bike Trail,Lounge,Sushi Restaurant
11,Roosevelt Island,0,Park,Sandwich Place,Coffee Shop,Playground,Dry Cleaner,Plaza,Café,Farmers Market,Supermarket,Metro Station
12,Upper West Side,0,Italian Restaurant,Bar,Wine Bar,Mediterranean Restaurant,Cosmetics Shop,Coffee Shop,Vegetarian / Vegan Restaurant,Indian Restaurant,Bakery,Bookstore
16,Murray Hill,0,Coffee Shop,Sandwich Place,Hotel,Japanese Restaurant,Italian Restaurant,Gym / Fitness Center,Gym,French Restaurant,Bar,Sushi Restaurant
18,Greenwich Village,0,Italian Restaurant,Sushi Restaurant,Clothing Store,Seafood Restaurant,Indian Restaurant,Cosmetics Shop,Café,French Restaurant,Chinese Restaurant,Cocktail Bar
19,East Village,0,Bar,Wine Bar,Ice Cream Shop,Pizza Place,Chinese Restaurant,Mexican Restaurant,Cocktail Bar,Ramen Restaurant,Speakeasy,Coffee Shop
22,Little Italy,0,Bakery,Café,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Clothing Store,Salon / Barbershop,Mediterranean Restaurant,Ice Cream Shop,Women's Store
23,Soho,0,Clothing Store,Boutique,Women's Store,Art Gallery,Shoe Store,Italian Restaurant,Men's Store,Mediterranean Restaurant,Furniture / Home Store,Bakery


In [88]:
Manhattan_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,1,Mexican Restaurant,Bakery,Deli / Bodega,Pizza Place,Thai Restaurant,Latin American Restaurant,French Restaurant,Steakhouse,Spanish Restaurant,Seafood Restaurant
13,Lincoln Square,1,Gym / Fitness Center,Theater,Plaza,Café,Concert Hall,Italian Restaurant,Performing Arts Venue,Park,Opera House,Indie Movie Theater
15,Midtown,1,Hotel,Coffee Shop,Cocktail Bar,Clothing Store,Theater,Japanese Restaurant,Spa,Sporting Goods Shop,Steakhouse,Bookstore
21,Tribeca,1,Italian Restaurant,Park,Café,American Restaurant,Spa,Boutique,Wine Bar,Gym,Greek Restaurant,Coffee Shop
25,Manhattan Valley,1,Pizza Place,Indian Restaurant,Coffee Shop,Yoga Studio,Café,Mexican Restaurant,Thai Restaurant,Deli / Bodega,Playground,Bar
28,Battery Park City,1,Park,Coffee Shop,Hotel,Memorial Site,Italian Restaurant,Wine Shop,Gym,Women's Store,Clothing Store,Playground
33,Midtown South,1,Korean Restaurant,Hotel,Dessert Shop,Japanese Restaurant,Hotel Bar,American Restaurant,Coffee Shop,Cosmetics Shop,Gastropub,Gym / Fitness Center
37,Stuyvesant Town,1,Bar,Playground,Park,Pet Service,Farmers Market,Basketball Court,Baseball Field,Gas Station,Fountain,Harbor / Marina


In [89]:
Manhattan_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,2,Coffee Shop,Pizza Place,Cosmetics Shop,Bar,Café,French Restaurant,Japanese Restaurant,Grocery Store,Yoga Studio,Spa


In [90]:
Manhattan_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,3,Sandwich Place,Coffee Shop,Discount Store,Yoga Studio,Diner,Steakhouse,Supplement Shop,Shopping Mall,Seafood Restaurant,Tennis Stadium
1,Chinatown,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Salon / Barbershop,Bubble Tea Shop,Bakery,Spa,Ice Cream Shop,Dumpling Restaurant
2,Washington Heights,3,Café,Mobile Phone Shop,Bakery,Grocery Store,Supermarket,Deli / Bodega,Coffee Shop,Park,Clothing Store,Chinese Restaurant
8,Upper East Side,3,Italian Restaurant,Exhibit,Art Gallery,Bakery,Coffee Shop,Gym / Fitness Center,Juice Bar,French Restaurant,Hotel,Spa
9,Yorkville,3,Italian Restaurant,Bar,Coffee Shop,Gym,Pizza Place,Sushi Restaurant,Wine Shop,Ice Cream Shop,Mexican Restaurant,Deli / Bodega
10,Lenox Hill,3,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Cosmetics Shop,Burger Joint,Gym,Gym / Fitness Center,Sporting Goods Shop,Thai Restaurant
20,Lower East Side,3,Coffee Shop,Pizza Place,Café,Sandwich Place,Art Gallery,Bakery,Ramen Restaurant,Chinese Restaurant,Japanese Restaurant,Cocktail Bar
24,West Village,3,Italian Restaurant,Cosmetics Shop,New American Restaurant,Cocktail Bar,American Restaurant,Wine Bar,Park,Jazz Club,Coffee Shop,Bakery
26,Morningside Heights,3,Park,Coffee Shop,American Restaurant,Bookstore,Food Truck,Tennis Court,Sandwich Place,Burger Joint,Deli / Bodega,Café
31,Noho,3,Italian Restaurant,French Restaurant,Art Gallery,Cocktail Bar,Coffee Shop,Rock Club,Pizza Place,Mexican Restaurant,Sushi Restaurant,Boutique


In [91]:
Manhattan_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,4,African Restaurant,Bar,Chinese Restaurant,Gym / Fitness Center,Seafood Restaurant,Public Art,Cosmetics Shop,Deli / Bodega,American Restaurant,Fried Chicken Joint
14,Clinton,4,Theater,Gym / Fitness Center,Italian Restaurant,Hotel,American Restaurant,Wine Shop,Coffee Shop,Spa,Sandwich Place,Food Court
17,Chelsea,4,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,Hotel,American Restaurant,Seafood Restaurant,Art Gallery,Theater
32,Civic Center,4,Italian Restaurant,Gym / Fitness Center,French Restaurant,Hotel,Sandwich Place,Sporting Goods Shop,American Restaurant,Bakery,Cocktail Bar,Coffee Shop
